
# Section 1 — Report Header & Hypothesis

**Report Title:**  API Data Report 

**Your Name:** Roman Bryenton

Date: 2025-10-07

Hypothesis

Accounts that include images in their posts receive a higher average number of likes per post than accounts that post only text.

On social media, visual content tends to attract more engagement with users on social media platforms because images tend to attract more attention more quickly than text. Images on social media capture and convey more information and emotional cues, which enhances user interaction.  On Bluesky, users scrolling through a feed are more likely to pause and engage (e.g., like or repost) with posts that feature compelling visuals. Therefore, accounts that frequently include images in their posts should see a measurable increase in engagement compared to those that rely solely on text-based updates.


Use a count model because the outcome (likes) is count data and often over-dispersed. The variables I will be using are things like post_id, post_time, and account_id. I will also use likes, reposts, and replies. 

    


**Planned Endpoints**
-app.bsky.feed.getPosts - to fetch posts by an account, so you can see whether each post contains images
Paramters used: uri of the post
What to extract: thread.post.record.embed.images

-app.bsky.feed.getLikes - to retrieve the number of likes on post
Parameter used- limit, Max number of likes to return per request. 
What to extract - post_id to see which posts the likes are for

-app.bsky.feed.getAuthorFeed- to see every post from a certain author
Parameter used- actor, The user’s decentralized ID (DID) or handle. 
What to extract feed[].post.uri- unique ID for each post.

**Any caveats in the response object**
-Like count may not be updated in real time
-Not all posts have embed fields
-Feed contains likes and reposts aswell

## Section 3 Data Collection

In [1]:
# imports
import requests 
import time
import json as js
import pandas as pd
from urllib.parse import urlencode


BASE_URL = "https://api.bsky.app/xrpc"


In [17]:

#getPosts
get_posts_url = f"{BASE_URL}/app.bsky.feed.getPosts"
params_posts = {"uris": post_uris}

posts_resp = requests.get(get_posts_url, headers=headers, params=params_posts, timeout=30)
print("getPosts Status:", posts_resp.status_code)
posts_data = posts_resp.json()
print("getPosts top-level keys:", list(posts_data.keys()))
print(json.dumps(posts_data, indent=2))


NameError: name 'post_uris' is not defined

In [18]:

#getAuthorFeed
author_did = "did:plc:abcd1234"  # or use handle
get_author_feed_url = f"{BASE_URL}/app.bsky.feed.getAuthorFeed"
params_author = {"actor": author_did, "limit": 5}  # limit optional

author_resp = requests.get(get_author_feed_url, headers=headers, params=params_author, timeout=30)
print("getAuthorFeed Status:", author_resp.status_code)
author_data = author_resp.json()
print("getAuthorFeed top-level keys:", list(author_data.keys()))
print(json.dumps(author_data, indent=2))

getAuthorFeed Status: 400
getAuthorFeed top-level keys: ['error', 'message']


NameError: name 'json' is not defined

## Build Data Frames

In [19]:
posts_list = feed_data.get("feed", [])
df_posts = pd.DataFrame([{
    "uri": post["post"]["uri"],
    "text": post["post"]["text"],
    "author": post["post"]["author"]["handle"]
} for post in posts_list])

print("Posts DataFrame:")
print(df_posts)


AttributeError: 'function' object has no attribute 'get'

In [20]:
likes_list = []

for uri in df_posts["uri"]:
    likes_url = f"{BASE_URL}/app.bsky.feed.getLikes"
    params_likes = {"uri": uri}
    likes_resp = requests.get(likes_url, headers=headers, params=params_likes, timeout=30)
    if likes_resp.status_code == 200:
        likes_data = likes_resp.json()
        likes_list.append({
            "uri": uri,
            "likes_count": len(likes_data.get("likes", []))
        })
    else:
        likes_list.append({"uri": uri, "likes_count": 0})

df_likes = pd.DataFrame(likes_list)

print("\nLikes DataFrame:")
print(df_likes)


NameError: name 'df_posts' is not defined

In [21]:
df_combined = pd.merge(df_posts, df_likes, on="uri", how="left")
print("\nCombined DataFrame:")
print(df_combined)


NameError: name 'df_posts' is not defined

## Section 5 Conclusion
- Imaged posts have varying like variety, some posts amass a ton of likes, while others don't get any
- Imaged posts have more extreme highs creating outliers
- Imaged posts tend to get higher like variety

  # Challenges
  I personally am still really struggling with the coding part of these assignments 